<center><H1>Code Snippet and Output</H3>

<h3>Importing all the modules and libraries required for the Project.<br><br>
There are libraries for<br> <br>
1. Creating Web Dash app<br><br>
2. Importing dataframe  <br><br>
3. Statistical Analysis<br><br>
4. Data Visualization<br><br>
5. Data Processing<br><br>

In [1]:
#For Web APP
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

#For Dataframe
import pandas as pd

#For Statistics Related Operations
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

import itertools
import numpy as np

#For Warnings
import warnings
warnings.filterwarnings("ignore")


C:\Users\vaibh\AppData\Local\Temp\ipykernel_1652\3557436012.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\vaibh\AppData\Local\Temp\ipykernel_1652\3557436012.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<H3>Initializing the Web App<br></H3>

In [2]:
app = dash.Dash()

<H3>There are three Datasets containing different informations.Datasets are imported with the help of pandas and  <br><br>
are merged with the help of left join.So, it can be easy for visualization and Statistical Analysis.<br></H3>

In [3]:
#Main DataFrame
train = pd.read_csv(r"C:\Users\vaibh\Machine Learning\DataSets\walmart-recruiting-store-sales-forecasting\train.csv")
stores = pd.read_csv(r"C:\Users\vaibh\Machine Learning\DataSets\walmart-recruiting-store-sales-forecasting\stores.csv")
stores = stores.merge(train,how='left')
df = stores
len(df.Date.unique())

143

In [4]:
train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [5]:
print(stores[stores.IsHoliday==True].Date.unique())


['2010-02-12' '2010-09-10' '2010-11-26' '2010-12-31' '2011-02-11'
 '2011-09-09' '2011-11-25' '2011-12-30' '2012-02-10' '2012-09-07']


<H3>Data for Box Plot are selected and columns which are not required are dropped.So, it can be easy to <br><br>
understand the data in highly effective manner and take decisions.</H3>

In [6]:
#For Box Plot
dfbox = df
features = dfbox.columns

<H3>All the columns which are not important for the SARIMA model are dropped and sorted on the basis of the Date.<br><br>
All the Sales Data are evaluated by taking mean of Sales on weekly basis.<br><br>
STL(Season-Trend) decomposition is used to decompose the Sales data into three components i.e. Trend, <br><br>
Seasonal and Residual.<br><br>
1. Trend Graph gives the overall idea about data, how it is being progressing over the time.<br><br>
2. Seasonality Graph gives the idea about the data in certain period over the years.<br><br>
3. Residual Graph gives information which is left after Trend and Seasonality graph.Thus, it is the noise in the <br><br>data.<br><br>

In [7]:
#For Decomposition AND Prediction Plot Data
dfarima = train

dfarima = dfarima.sort_values('Date')
dfarima['Date'] = pd.to_datetime(dfarima['Date'], errors='coerce')

dfarima = dfarima.set_index('Date')
sales = dfarima['Weekly_Sales'].groupby(['Date']).mean()
print(type(sales))
salesarima = sales # For Prediction

sales = sales.to_frame() # For Decomposition
sales.reset_index(level=0, inplace=True)

#For Decomposition
salesdecompose = sales.set_index('Date')
resultdecompose = STL(salesdecompose,robust = True).fit()


<class 'pandas.core.series.Series'>


<H3>Graph of the Observed Sales Data is plotted. Also the graph of  different components are plotted i.e.Trend,<br><br> Seasonal and Residual. 
After decomposing the data, graphs are developed with the same layout.<br><br>
On X-axis, we have "Date"<br><br>
On Y-axis, we have "Values"
    
These graphs help us to understand the data and to take further decisions in the process.


In [8]:
#Graph For Decompostion
#layout for the Graph
layoutdecomposition=go.Layout(hovermode='x',spikedistance=1000,font_family='Sailec',
    xaxis=dict(title="Date",showspikes=True, spikethickness=2,spikedash='dot',spikecolor='#999999',spikemode='across'),
               title=dict(text='Residual Graph',y=0.9,x=0.5,xanchor= 'center',yanchor='top'),
               paper_bgcolor='#434343',plot_bgcolor='#000000',font_color='white',title_font_size=30,font_size=16)
    
#Observed Graph      
oberved = go.Scatter(x=sales['Date'],y=sales['Weekly_Sales'],mode='lines',                                    
             marker=dict(size=2,color='#0AC0C0',line=dict(width=2)))
observed_graph=go.Figure(data=oberved,layout=layoutdecomposition)
observed_graph.update_layout(title=dict(text='Observed Graph',y=0.9,x=0.5,xanchor= 'center',yanchor='top'))
   
#Trend Graph    
trend = go.Scatter(x=resultdecompose.trend.index,y=resultdecompose.trend,mode='lines',
             marker=dict(size=2,color='#0AC0C0',line=dict(width=2)))
trend_graph = go.Figure(data=trend,layout=layoutdecomposition)
trend_graph.update_layout(title=dict(text='Trend Graph',y=0.9,x=0.5,xanchor= 'center',yanchor='top'))

#Seasonal Graph
seasonal = go.Scatter(x=resultdecompose.seasonal.index,y=resultdecompose.seasonal,mode='lines',
             marker=dict(size=2,color='#0AC0C0',line=dict(width=2)))
seasonal_graph = go.Figure(data=seasonal,layout=layoutdecomposition)
seasonal_graph.update_layout(title=dict(text='Seasonal Graph',y=0.9,x=0.5,xanchor= 'center',yanchor='top'))    
    
#Residual Graph
residual = go.Scatter(x=resultdecompose.resid.index,y=resultdecompose.resid,mode='lines',
             marker=dict(size=2,color='#0AC0C0',line=dict(width=2)))
residual_graph = go.Figure(data=residual,layout=layoutdecomposition)

<H3>In this section, we select the set of values for the prediction model(SARIMA);with the help of itertools , it creates <br><br>cartesian product for the given set of parameters.Then these set of paranmeters are run through the model that <br><br>give different results on running.These set of results are stored in the dataframe. Then, we find the set of<br><br> parameters  corresponding to the minimum value of result.<br><br><br>

In [9]:
#Forecasting Section
p = d = q = range(0,2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]
resultdf=[]
resultaicdf=[]
#For Selecting Appropriate  Set of Parameters for Forecasting
for param in pdq:
    for param_seasonal in seasonal_pdq:
            mod = sm.tsa.statespace.SARIMAX(salesarima,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,
                                                enforce_invertibility=False)
            results = mod.fit()

            resultdf.append('SARIMAX{}x{}52'.format(param,param_seasonal))
            resultaicdf.append('{}'.format(results.aic))

dfresultsaic=pd.DataFrame({'Set Values':resultdf,
'Results':resultaicdf
})            
dfresultsaic['Results'] = pd.to_numeric(dfresultsaic['Results'])
print(dfresultsaic[dfresultsaic.Results == dfresultsaic.Results.min()])

                          Set Values     Results
31  SARIMAX(0, 1, 1)x(1, 1, 1, 52)52  558.807058


<H3>From previous section it was easy to deduct the set of parameters and these set of parameters can be used for <br><br>our SARIMA Model.

In [10]:
mod = sm.tsa.statespace.SARIMAX(salesarima,order=(0,1,1),seasonal_order=(1,1,1,52),enforce_stationarity=False,
enforce_invertibility = False)
results = mod.fit()


<h3>After creating a model sales data can easily be plotted over the last six months provided in the dataset. First<br><br> the required datsets are processed, so it can be used for developing traces.

In [11]:
#Forecasting Graph Data for small period of time
pred = results.get_prediction(start=pd.to_datetime('2012-01-06'),dynamic=True)
pred_ci = pred.conf_int()

ax = salesarima['2010-02-05':] #For Both Forecasting Graph
pred = pred.predicted_mean
ax = ax.to_frame()
pred = pred.to_frame()

ax.reset_index(level=0, inplace=True)
pred.reset_index(level=0, inplace=True)
#For renaming Column
pred = pred.rename(columns={'index': 'Date'})
pred_ci.columns.values[0]='Date'
ax

,Date,Weekly_Sales
0,2010-02-05,16836.121997
1,2010-02-12,16352.056032
2,2010-02-19,16216.658979
3,2010-02-26,14899.549688
4,2010-03-05,15921.015727
...,...,...
138,2012-09-28,14765.327279
139,2012-10-05,15983.413747
140,2012-10-12,15427.596739
141,2012-10-19,15295.732397


<H3>Traces are being developed for the last six months of the dataset.So, it can be easily compared with the <br><br>actual data and RMS Error can be calculated easily.  

In [12]:
#Traces for small period in future
traceax = go.Scatter(x=ax['Date'],y=ax['Weekly_Sales'],mode='lines',name='Observed',line_color='rgb(10, 192, 192)')

tracepred = go.Scatter(x=pred['Date'],y=pred['predicted_mean'],mode='lines',name='Predicted')

traceupperpred = go.Scatter(x=pred['Date'],y=pred_ci.iloc[:,1],mode='lines',name='Predicted',showlegend=False,
                         fillcolor='rgba(169,169,169,0.5)',line_color='rgba(169,169,169,0)',hoverinfo='skip')

tracelowerpred = go.Scatter(x=pred['Date'],y=pred_ci.iloc[:,0],mode='lines',name='Predicted',fill='tonexty',showlegend=False,
                         fillcolor='rgba(169,169,169,0.5)',line_color='rgba(169,169,169,0)',hoverinfo='skip')

datapred = [traceax,tracepred,traceupperpred,tracelowerpred]

layoutpred = go.Layout(hovermode='x',spikedistance=1000,font_family='Sailec',
    xaxis=dict(title='Date',showspikes=True, spikethickness=2,spikedash='dot',spikecolor='#999999',spikemode='across',),
    yaxis=dict(title='Average Weekly Sales'),paper_bgcolor='#434343',plot_bgcolor='#000000',font_color='white',font_size=16)

figpred=go.Figure(data=datapred,layout=layoutpred)


<H3>Data for the model are processed over the larger period of time, so it can be used for developing traces.

In [13]:
#Forecasting Graph Data for large period of time
pred_uc500 = results.get_forecast(steps=500)
pred_ci500 = pred_uc500.conf_int()

pred_uc500 = pred_uc500.predicted_mean
pred_uc500 = pred_uc500.to_frame()
pred_uc500.reset_index(level=0, inplace=True)
pred_uc500.columns.values[0]='Date'

<H3>Traces are being developed after the end of data i.e. over the larger period over time(years).So, it can be<br><br>
easily predicted on the basis of historical data and take data driven decisions.

In [14]:
#Traces for large period in future

tracepred_uc500 = go.Scatter(x=pred_uc500['Date'],y=pred_uc500['predicted_mean'],mode='lines',name='Predicted')

traceupperpred_uc500 = go.Scatter(x=pred_uc500['Date'],
                     y=pred_ci500.iloc[:,1],mode='lines',name='Predicted',showlegend=False,
                     fillcolor='rgba(169,169,169,0.5)',line_color='rgba(169,169,169,0)',hoverinfo='skip')

tracelowerpred_uc500 = go.Scatter(x=pred_uc500['Date'],
                     y=pred_ci500.iloc[:,0],mode='lines',name='Predicted',fill='tonexty',showlegend=False,
                     fillcolor='rgba(169,169,169,0.5)',line_color='rgba(169,169,169,0)',hoverinfo='skip')

datapred_uc500 = [traceax,tracepred_uc500,traceupperpred_uc500,tracelowerpred_uc500]

figpred_uc500 = go.Figure(data=datapred_uc500,layout=layoutpred)


<H3>For every regressive model Root Mean Square(RMS) error is calculated to find out about the efficiency<br><br> of the model.<br><br>
RMS Error is given by:  

In [15]:
#Root Mean Square Error
truthax = salesarima['2012-01-06':]
truthax = truthax.to_frame()
truthax.reset_index(level=0, inplace=True)
del truthax['Date']

errorpred = pred
del errorpred['Date']

error = np.square(errorpred['predicted_mean']-truthax['Weekly_Sales'])
stderror = error.mean()
rmseerror = round(np.sqrt(stderror), 2)



<H3>This is the main section of our app. Layout of the app is developed.This section conatins the result of our <br><br>statistical computations and graphs are displayed.This section only contains the result of our computations as<br><br> user does is not concerned with process , only with the result. Main Purpose of this section to express the graph<br><br> in highly effective and attractive manner. Graphs should be easy to understand and must have proper layout.<br><br>This section will act as front layer for the user who want to take data driven decisions.

In [16]:
#APP SECTION
app.layout = html.Div([
    
    #Styling the heading
    html.Div([
    html.Div(html.Img(src=app.get_asset_url("WalmartSpark.png"),
                      style=dict(height='10%',width='7%',marginBottom='-2%',marginTop='2%',marginLeft='47%'),)),
    
    html.H1(children='Walmart Sales Analysis',style=dict(fontSize='48px',color='white',textAlign='center')),
    
    ],style=dict(backgroundColor='',margin='0%')),
    
    #DATA Visualization Section
    html.Div([html.H1(children='''Data Visualization - Box Plot''',style=dict(textAlign='center',fontSize='40px'))],
            style=dict(backgroundColor='#1F2833',marginTop='2%',color='white')),
    
    # Box Plot
    html.Div([
    html.Div([html.H2("X-axis:")],style=dict(width='50%',display='inline-block',fontSize='115%',color='white')),
    html.Div([html.H2("Y-axis:")],style=dict(width='50%',display='inline-block',fontSize='115%',color='white')),
    
    #DropDown 1
    html.Div([
    dcc.Dropdown(
        id = 'x-axis', 
        options = [{'value': x, 'label': x} 
                 for x in features],
        value = 'Store', 
    )],style = dict(width='50%',display='inline-block')),
    
    #DropDown 2
    html.Div([
    dcc.Dropdown(
        id = 'y-axis', 
        options = [{'value': x, 'label': x} 
                 for x in features],
        value = 'Weekly_Sales',  
    )],style = dict(width='50%',display='inline-block')),
    
    #Box-Graph
    html.Div([dcc.Graph(id="box-plot")])
    ],style = dict(width='98%',marginLeft='1%')),   
    
    #SARIMA SECTION
    #Decomposition Section
    html.Div([html.H1(children='''SARIMA SECTION''',style=dict(textAlign="center",fontSize= "40px"))]
             ,style=dict(backgroundColor='#1F2833',margin='0%',color='white')),
    
    html.Div([html.H1(children='''Decomposition Of Sales DATA ''',
              style=dict(textAlign="center",fontSize= "30px",color='white'))]),
    
    html.Div([
    #Observed Graph    
    html.Div([dcc.Graph(figure=observed_graph,config=dict(displayModeBar=False))],
             style=dict(display='inline-block',width='49%',marginLeft='1%')),
    
    #Trend Graph
    html.Div([dcc.Graph(figure=trend_graph,config=dict(displayModeBar=False))],
             style=dict(display='inline-block',width='48%',marginLeft='1%'))]),
    
    html.Div([
    #Seasonal Graph
    html.Div([dcc.Graph(figure=seasonal_graph,config=dict(displayModeBar=False))],
             style=dict(display='inline-block',width='49%',marginLeft='1%')),
        
    #Residual Graph
    html.Div([dcc.Graph(figure=residual_graph,config=dict(displayModeBar=False))],
             style=dict(display='inline-block',width='48%',marginLeft='1%'))],style=dict(marginTop='4%')),
     
    html.Br(),html.Br(),html.Br(),
    #Predicted Graph
    html.H1(children='''Forecasting of Walmart Sales Data''',
            style=dict(textAlign='center',color='white',fontSize='40px',backgroundColor='#1F2833')),
   
    html.H2(children='''Forecasting Over 6 Months''',
            style=dict(textAlign='center',fontSize='36px',color='white')),
    html.Div([dcc.Graph(figure=figpred)],style=dict(width='98%',marginLeft='1%')),
    
    html.H2(children='''Forecasting Over next 10 Years''',
            style=dict(textAlign='center',fontSize='36px',color='white')),
    html.Div([dcc.Graph(figure=figpred_uc500)],style=dict(width='98%',marginLeft='1%')),
    html.H5(children='''Note : All the values used in the line graph is based on average weekly sales across all 
    departments and stores on a particular day.i.e.Average Weekly Sales of all departments across
    all stores on a particular day. 
    Formula Used for Average Weekly Sales (Departments) on a particular day =
    Total Sales/Total number weekly sales column present on that date ,''',
            style=dict(textAlign='center',color='white')),
    html.Div([
        html.H1('The Root Mean Square Error in the Forecasting Model is {}'.format(rmseerror),
           style=dict(color='white',fontSize='36px')),
    ],style=dict(marginLeft='1%'))
    
],style=dict(backgroundImage='linear-gradient(#000000,#2d3436)',margin='-8px')) 


<H3>Box Plot is developed according to the user preference , so it can be easily used.

In [17]:
#Box Plots Callback
@app.callback(
    Output("box-plot", "figure"), 
    [Input("x-axis", "value"), 
     Input("y-axis", "value")])

def generate_chart(x, y):
    fig = px.box(df, x=x, y=y,color=x,hover_data=["Date","Store"])
    fig.update_layout(font_family='Sailec',paper_bgcolor='#434343',plot_bgcolor='#000000',font_color='white',font_size=16)
    return fig


<H3>In this section app is run, and user can easily see the results.

In [ ]:
if __name__=='__main__':
    app.run_server(port=8080)
    

Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jun/2022 20:21:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/deps/react@16.v2_5_1m1656512868.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_5_1m1656512868.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_5_1m1656512867.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_5_0m1656512868.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/dash_table/bundle.v5_1_3m1656512867.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_3m1656512868.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2022 20:21:14] "GET /_dash-component-suites/dash/de

509.884864050486
